# Performance of JAX vs PyTorch

Observataions:
- it seems that JAX with JIT is fatser than Pytorch JIT.Script
- for my toy kernel using the where() method, JAX is 20-30x faster PyTorch
- Jax JIT can be used for class method as well. 
- torch jit script works only for functions and subclasses of nn.Module
- GPU version is generall slower for my micro kernel and for both Jax and Torch
- Jax GPU version is also faster than Torch
- There is not much overhead when using utility jit kenrel inside the class method

An useful link https://www.kaggle.com/code/grez911/performance-of-jax-vs-pytorch

In [ ]:
%load_ext memory_profiler

In [ ]:
N = 10000

## Jax

In [ ]:
%env JAX_ENABLE_X64=1
%env JAX_PLATFORM_NAME=cpu
device='cpu'

In [ ]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

In [ ]:
def jax_kernel(x):
    return jnp.sum(jnp.where(x > 0.5, x, x**2))

In [ ]:
key = random.PRNGKey(0)
x = random.uniform(key, (N,))
x

In [ ]:
%timeit jax_kernel(x)

In [ ]:
jit_jax_kernel = jit(jax_kernel)

%timeit jit_jax_kernel(x).block_until_ready()

In [ ]:
derivative_jax_kernel = grad(jax_kernel)

%timeit derivative_jax_kernel(x).block_until_ready()

In [ ]:
derivative_jit_jax_kernel = jit(derivative_jax_kernel)

%timeit derivative_jit_jax_kernel(x).block_until_ready()

In [ ]:
%memit jit_jax_kernel(x)

In [ ]:
%memit derivative_jit_jax_kernel(x)

## Torch

In [ ]:
import torch

In [ ]:
def torch_kernel(x):
    return torch.sum(torch.where(x > 0.5, x, x**2))

In [ ]:
x = torch.rand((N,), requires_grad=True)
# x

In [ ]:
%timeit torch_kernel(x)

In [ ]:
jit_torch_kernel = torch.jit.script(torch_kernel)

%timeit jit_torch_kernel(x)

In [ ]:
%timeit torch.autograd.grad(torch_kernel(x), [x])

In [ ]:
%timeit torch.autograd.grad(jit_torch_kernel(x), [x])

In [ ]:
%memit jit_torch_kernel(x)

In [ ]:
%memit torch.autograd.grad(jit_torch_kernel(x), [x])

### Further experiments

#### Jax JIT

In [ ]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

In [ ]:
x = random.uniform(key, (1000,3))
x.dtype

In [ ]:
def F(x, y):
    return x.sum() + 2*y.sum()

In [ ]:
dF = jit(grad(F, argnums=1))

In [ ]:
dF(x, x)

In [ ]:
@jit
def F(x, y):
    sum_x  = x.sum()
    print(f'{x=}')
    sum_y = 2*y.sum()
    print(f'{sum_y=}')
    print(f'{x.shape=}')
    return sum_x + sum_y

In [ ]:
F(x, x)

#### torch scriptted function

In [ ]:
import torch
from torch import Tensor 

@torch.jit.script
def cutoff_function(r: Tensor, r_cutoff: float) -> Tensor:
    cfn = torch.tanh(1.0 - r / r_cutoff).pow(3)
    return torch.where(r < r_cutoff, cfn, torch.zeros_like(r))

r = torch.tensor(1.0) 
r_cutoff = 2.0
cutoff_function(r, r_cutoff)

In [ ]:
# scripted_call.graph

In [ ]:
%timeit cutoff_function(r, r_cutoff)

In [ ]:
%timeit cutoff_function(r, r_cutoff)

#### Cutoff function class

In [ ]:
import torch
import jax.numpy as jnp
import numpy as onp
from jax import jit, jacfwd, grad
from functools import partial
from torch import Tensor

In [ ]:
_LIB = jnp # torch
    
class CutoffFunction:

    _TANH_PRE = ((_LIB.e + 1 / _LIB.e) / (_LIB.e - 1 / _LIB.e))**3

    def __init__(self, r_cutoff: float, cutoff_type: str = "tanh"):
        self.r_cutoff = r_cutoff
        self.cutoff_type = cutoff_type.lower()
        self.inv_r_cutoff = 1.0 / self.r_cutoff
        # Set cutoff type function
        try:
            self.cfn = getattr(self, f"{self.cutoff_type}")
        except AttributeError:
            logger.error(
                f"'{self.__class__.__name__}' has no cutoff function '{self.cutoff_type}'",
                exception=NotImplementedError,
            )

    @partial(jit, static_argnums=(0,))
    def __call__(self, r: Tensor) -> Tensor:
        return _LIB.where(r < self.r_cutoff, self.cfn(r), _LIB.zeros_like(r))

    def hard(self, r: Tensor) -> Tensor:
        return _LIB.ones_like(r)

    def tanhu(self, r: Tensor) -> Tensor:
        return _LIB.tanh(1.0 - r * self.inv_r_cutoff)**3

    def tanh(self, r: Tensor) -> Tensor:
        return self._TANH_PRE * _LIB.tanh(1.0 - r * self.inv_r_cutoff)**3

    def cos(self, r: Tensor) -> Tensor:
        return 0.5 * (_LIB.cos(math.pi * r * self.inv_r_cutoff) + 1.0)

    def exp(self, r: Tensor) -> Tensor:
        return _LIB.exp(1.0 - 1.0 / (1.0 - (r * self.inv_r_cutoff) ** 2))

    def poly1(self, r: Tensor) -> Tensor:
        return (2.0 * r - 3.0) * r**2 + 1.0

    def poly2(self, r: Tensor) -> Tensor:
        return ((15.0 - 6.0 * r) * r - 10) * r**3 + 1.0

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(r_cutoff={self.r_cutoff}, cutoff_type='{self.cutoff_type}')"


In [ ]:
r = jnp.array(2.0) # torch.tensor(1.0)
r_cutoff = 3.0

cfn = CutoffFunction(r_cutoff)
cfn(r)

In [ ]:
%timeit cfn(r)

In [ ]:
%timeit cfn(r)

In [ ]:
cfn(r)

In [ ]:
cfn(r*0.5)

In [ ]:
cfn.r_cutoff = 1000.0

#### Class method vs kernel function

In [ ]:
import jax.numpy as jnp
import numpy as onp
from jax import jit, jacfwd, grad
from functools import partial

In [ ]:
@jit
def _cutoff_function(r: jnp.ndarray, r_cutoff: float) -> jnp.ndarray:
    cfn = jnp.tanh(1.0 - r / r_cutoff)**3
    return jnp.where(r < r_cutoff, cfn, jnp.zeros_like(r))

In [ ]:
class CutoffFunction:
    def __init__(self, r_cutoff: float):
        self.r_cutoff = r_cutoff
        
    def __call__(self, r):
        return _cutoff_function(r, self.r_cutoff)

In [ ]:
r = jnp.array(1.0) # torch.tensor(1.0)
r_cutoff = 2.0

cfn = CutoffFunction(r_cutoff)
cfn(r)

In [ ]:
%timeit _cutoff_function(r, r_cutoff)

In [ ]:
%timeit cfn(r)